## Comparing code
---

In [1]:
########################################  readme  #####################################
# comparing the descriptors between new version and old version
# you can change targetFileName variable(line 25) to run the code with different file
# if the descriptors are the same, then you can see OK and value
#######################################################################################

import hwi_igraph_testing_main as ig
import igraph_testing_main as ig2
import importlib
import descriptors as ds
import os
import time
import matplotlib.pyplot as plt
import tracemalloc

importlib.reload(ig)
importlib.reload(ig2)

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/p1descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon = 1e-5

targetFileName = '0.25'    
loop_cnt = 1

def run_test(lib, test_file):
    total_graph_time = 0
    tracemalloc.start()
    graph_start = time.time()
    g, *_ = lib.generateGraph(data_path + test_file + ".txt")
    _stats = tracemalloc.get_traced_memory()
    graph_end = time.time()
    tracemalloc.stop()

    graph_mem = _stats[1] - _stats[0]
    stats = ds.desciptors(g)
    graph_time = graph_end - graph_start
    return stats, graph_time, graph_mem

# Run and compare for both ig and ig2
for test_file in test_files:
    if targetFileName not in test_file:
        continue

    print(f"--- {test_file} ---")

    # Run for ig
    stats_ig, time_ig, mem_ig = run_test(ig, test_file)

    # Run for ig2
    stats_ig2, time_ig2, mem_ig2 = run_test(ig2, test_file)

    print("\n[Descriptor Comparison]")
    keys = set(stats_ig.keys()).union(set(stats_ig2.keys()))
    for key in sorted(keys):
        v1 = stats_ig.get(key, None)
        v2 = stats_ig2.get(key, None)
        if v1 is None or v2 is None:
            print(f"{key}: missing in one version -> new: {v1}, prev: {v2}")
        elif abs(v1 - v2) < epsilon:
            print(f"{key}: OK -> new : {v1}, prev: {v2}")
        else:
            print(f"{key}: DIFFERENT -> new: {v1}, prev: {v2}")

    # Time and memory comparison
    print("\n[Performance Comparison]")
    print(f"Graph Generation Time - new: {time_ig:.6f}s, prev: {time_ig2:.6f}s")
    print(f"Graph Memory Usage     - new: {mem_ig} bytes, prev: {mem_ig2} bytes")
    print(f"Descriptor Time        - new: {stats_ig['time']:.6f}s, prev: {stats_ig2['time']:.6f}s")
    print(f"Descriptor Memory      - new: {stats_ig['mem']} bytes, prev: {stats_ig2['mem']} bytes")
    print()

--- morphology_resize_0.25x ---

[Descriptor Comparison]
ABS_f_D: OK -> new : 0.53836, prev: 0.53836
CT_f_conn_A_Ca: OK -> new : 1.0, prev: 1.0
CT_f_conn_D_An: OK -> new : 0.287961, prev: 0.287961
CT_n_A_adj_Ca: OK -> new : 29, prev: 29
CT_n_D_adj_An: OK -> new : 48, prev: 48
STAT_CC_A: OK -> new : 1, prev: 1
STAT_CC_A_Ca: OK -> new : 1, prev: 1
STAT_CC_D: OK -> new : 7, prev: 7
STAT_CC_D_An: OK -> new : 3, prev: 3
STAT_e: DIFFERENT -> new: 561, prev: 596
STAT_n: OK -> new : 3780, prev: 3780
STAT_n_A: OK -> new : 1745, prev: 1745
STAT_n_D: OK -> new : 2035, prev: 2035
mem: DIFFERENT -> new: 29863, prev: 29959
time: DIFFERENT -> new: 0.39464282989501953, prev: 0.38310718536376953

[Performance Comparison]
Graph Generation Time - new: 2.125344s, prev: 2.111962s
Graph Memory Usage     - new: 4317086 bytes, prev: 4072006 bytes
Descriptor Time        - new: 0.394643s, prev: 0.383107s
Descriptor Memory      - new: 29863 bytes, prev: 29959 bytes



## individual code : new_version
---

In [3]:
import hwi_igraph_testing_main as ig
import importlib
import descriptors as ds
import os
import time
import matplotlib.pyplot as plt
import tracemalloc

importlib.reload(ig)  

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/p1descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

targetFileName = '0.25'
loop_cnt = 1

    
for test_file in test_files:
    if targetFileName not in test_file:
        continue

    import time
    total_graph_time = 0
    for i in range(loop_cnt):

        # g = ig.generateGraph(data_path + test_file + ".txt")
        tracemalloc.start()
        graph_start = time.time()
        g,is_2D, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca= ig.generateGraph(data_path + test_file + ".txt")
        _stats = tracemalloc.get_traced_memory()
        graph_end = time.time()     
        tracemalloc.stop()
        graph_mem = _stats[1]-_stats[0]  
        stats = ds.desciptors(g)
        total_graph_time += graph_end - graph_start
        #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")

    with open(descriptors_path + "p1descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
    descriptor_time = stats["time"]
    descriptor_mem = stats["mem"]

    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = total_graph_time/loop_cnt
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



morphology_resize_0.25x Results
STAT_n passed
STAT_e failed - 561 is not the same as expected 613
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_n_D_adj_An failed - 48 is not the same as expected 49
CT_n_A_adj_Ca failed - 29 is not the same as expected 30
Total time to calculate graph: 2.1280808448791504 second(s)
Total time to calculate descriptors: 0.36905598640441895 second(s)
Peak memory usage for graph generation: 4317680 bytes
Peak memory usage for descriptor calculation: 29959 bytes
{'STAT_n': 3780, 'STAT_e': 561, 'STAT_n_D': 2035, 'STAT_n_A': 1745, 'STAT_CC_D': 7, 'STAT_CC_A': 1, 'STAT_CC_D_An': 3, 'STAT_CC_A_Ca': 1, 'ABS_f_D': 0.53836, 'CT_f_conn_D_An': 0.287961, 'CT_f_conn_A_Ca': 1.0, 'CT_n_D_adj_An': 48, 'CT_n_A_adj_Ca': 29, 'time': 0.36905598640441895, 'mem': 29959}



## individual code : previous version
---

In [ ]:
import igraph_testing_main as ig
import importlib
import descriptors as ds
import os
import time
import matplotlib.pyplot as plt
import tracemalloc

importlib.reload(ig)  

current_dir = os.getcwd()
data_path = f"{current_dir}/graspi_igraph/data/"
descriptors_path = f"{current_dir}/graspi_igraph/p1descriptors/"
test_files = [os.path.splitext(file)[0] for file in os.listdir(data_path)]
epsilon=1e-5

times = []
mems = []
time_mem_stats = {}

targetFileName = '0.25'


for test_file in test_files:
    if targetFileName not in test_file:
        continue

    import time
    total_graph_time = 0
    for i in range(loop_cnt):

        # g = ig.generateGraph(data_path + test_file + ".txt")
        tracemalloc.start()
        graph_start = time.time()
        g,is_2D, black_vertices, white_vertices, black_green, black_interface_red, white_interface_blue, dim, interface_edge_comp_paths, shortest_path_to_red, shortest_path_to_blue, CT_n_D_adj_An, CT_n_A_adj_Ca= ig.generateGraph(data_path + test_file + ".txt")
        _stats = tracemalloc.get_traced_memory()
        graph_end = time.time()     
        tracemalloc.stop()
        graph_mem = _stats[1]-_stats[0]  
        stats = ds.desciptors(g)
        total_graph_time += graph_end - graph_start
        #ig.visual2D(g, 'graph')

    print(f"{test_file} Results")
    with open(descriptors_path + "p1descriptors." + test_file + ".log") as f:
        for line in f:
            stat = line.strip().split(" ")
            try:
                # if stats.get(stat[0], -1) == int(stat[1]):
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != int(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")
            except ValueError:
                if abs(stats.get(stat[0], -1) - float(stat[1])) < epsilon:
                    print(f"{stat[0]} passed")
                elif stats.get(stat[0], -1) != -1 and stats.get(stat[0], -1) != float(stat[1]):
                    print(f"{stat[0]} failed - {stats.get(stat[0])} is not the same as expected {stat[1]}")

    descriptor_time = stats["time"]
    descriptor_mem = stats["mem"]

    times.append(descriptor_time)
    mems.append(descriptor_mem)

    graph_time = total_graph_time/loop_cnt
    print(f"Total time to calculate graph: {graph_time} second(s)")
    print(f"Total time to calculate descriptors: {descriptor_time} second(s)")
    print(f"Peak memory usage for graph generation: {graph_mem} bytes")
    print(f"Peak memory usage for descriptor calculation: {descriptor_mem} bytes")
    print(stats)
    print("")
    time_mem_stats[test_file] = {"graph_time": graph_time, "descriptor_time": descriptor_time,  "graph_mem":graph_mem, "descriptor_mem": descriptor_mem}



data_4x3x2 Results
STAT_n passed
STAT_e failed - 12 is not the same as expected 24
STAT_n_D passed
STAT_n_A passed
STAT_CC_D passed
STAT_CC_A passed
STAT_CC_D_An passed
STAT_CC_A_Ca passed
ABS_f_D passed
CT_f_conn_D_An passed
CT_f_conn_A_Ca passed
CT_n_D_adj_An failed - 6 is not the same as expected 7
CT_n_A_adj_Ca failed - 6 is not the same as expected 7
Total time to calculate graph: 0.024044036865234375 second(s)
Total time to calculate descriptors: 0.004251241683959961 second(s)
Peak memory usage for graph generation: 28219 bytes
Peak memory usage for descriptor calculation: 389 bytes
{'STAT_n': 24, 'STAT_e': 12, 'STAT_n_D': 12, 'STAT_n_A': 12, 'STAT_CC_D': 1, 'STAT_CC_A': 1, 'STAT_CC_D_An': 1, 'STAT_CC_A_Ca': 1, 'ABS_f_D': 0.5, 'CT_f_conn_D_An': 1.0, 'CT_f_conn_A_Ca': 1.0, 'CT_n_D_adj_An': 6, 'CT_n_A_adj_Ca': 6, 'time': 0.004251241683959961, 'mem': 389}



--- data_4x3x2 ---

[Descriptor Comparison]
ABS_f_D: OK -> new : 0.5, prev: 0.5
CT_f_conn_A_Ca: OK -> new : 1.0, prev: 1.0
CT_f_conn_D_An: OK -> new : 1.0, prev: 1.0
CT_n_A_adj_Ca: OK -> new : 6, prev: 6
CT_n_D_adj_An: OK -> new : 6, prev: 6
STAT_CC_A: OK -> new : 1, prev: 1
STAT_CC_A_Ca: OK -> new : 1, prev: 1
STAT_CC_D: OK -> new : 1, prev: 1
STAT_CC_D_An: OK -> new : 1, prev: 1
STAT_e: OK -> new : 12, prev: 12
STAT_n: OK -> new : 24, prev: 24
STAT_n_A: OK -> new : 12, prev: 12
STAT_n_D: OK -> new : 12, prev: 12
mem: OK -> new : 389, prev: 389
time: DIFFERENT -> new: 0.004912853240966797, prev: 0.004218101501464844

[Performance Comparison]
Graph Generation Time - new: 0.021146s, prev: 0.020020s
Graph Memory Usage     - new: 31517 bytes, prev: 26332 bytes
Descriptor Time        - new: 0.004913s, prev: 0.004218s
Descriptor Memory      - new: 389 bytes, prev: 389 bytes

